# Этап 3: Ручное квантование в INT8 (Naive Quantization)

На этом этапе мы реализуем простейшее **линейное квантование**. 
Идея проста: мы берем диапазон весов слой за слоем и отображаем их из 32-битных чисел (float32) в 8-битные целые числа (int8).

### Формула квантования:
$$Q(x) = \text{round}\left(\frac{x}{S} + Z\right)$$
где:
- $S$ (Scale) — масштаб.
- $Z$ (Zero-point) — точка нуля.

В нашем примере мы будем использовать **Symmetric Quantization** (для простоты), где $Z=0$.

In [ ]:
import torch
import copy
from src.model import GPTLanguageModel, device, get_batch, estimate_loss

# 1. Загружаем оригинал
model_fp32 = GPTLanguageModel().to(device)
model_fp32.load_state_dict(torch.load('model_ckpt.pt', map_location=device))
model_fp32.eval()

def get_val_loss(mdl):
    with torch.no_grad():
        losses = estimate_loss(mdl)
    return losses['val'].item()

base_loss = get_val_loss(model_fp32)
print(f"Исходный Loss (FP32): {base_loss:.4f}")

### 2. Реализация линейного квантования
Мы напишем функцию, которая имитирует потерю точности при переходе в INT8.

In [ ]:
def quantize_tensor_int8(x):
    # Вычисляем масштаб (Scale)
    # Для 8 бит диапазон [-128, 127]
    x_max = x.abs().max().item()
    if x_max == 0: return x, 1.0
    
    scale = x_max / 127.0
    
    # Квантуем: float -> int8 -> float (dequantize)
    # Мы делаем Fake Quantization: оставляем тензор во флоатах, но со значениями, кратными шагу
    q_x = torch.round(x / scale).clamp(-128, 127)
    dq_x = q_x * scale
    
    return dq_x

# Создаем копию модели для квантования
model_int8 = copy.deepcopy(model_fp32)

with torch.no_grad():
    for name, param in model_int8.named_parameters():
        if 'weight' in name and param.dim() > 1:
            print(f"Квантуем слой: {name}")
            param.copy_(quantize_tensor_int8(param.data))

new_loss = get_val_loss(model_int8)
print(f"\nLoss после INT8 квантования: {new_loss:.4f}")
print(f"Деградация (Delta Loss): {new_loss - base_loss:.4f}")

### 3. Проверка результата генерацией
Посмотрим, не начал ли квантованный Шекспир нести полную чепуху.

In [ ]:
from src.model import decode

context = torch.zeros((1, 1), dtype=torch.long, device=device)
print("--- Output AFTER INT8 Quantization ---")
print(decode(model_int8.generate(context, max_new_tokens=200)[0].tolist()))